In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from povms.library.pm_sim_implementation import ClassicalShadows, RandomizedPMs
from povms.post_processor.povm_post_processor import POVMPostprocessor
from povms.sampler.povm_sampler import POVMSampler
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import (
    Operator,
    SparsePauliOp,
    Statevector,
    random_hermitian,
)
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime import Session

# Random Circuit

We first llok at a random the 2-qubit circuit, with depth 3.

In [3]:
n_qubit = 3
n_cicuits = 5
np.random.seed(10)
seeds= np.random.randint(0,10000000,size=n_cicuits)
qc_random = [random_circuit(num_qubits=n_qubit, depth = 3, measure=False, seed=seed) for seed in seeds]
qc_random[0].draw()

┌──────┐┌──────┐                                                
q_0: ─────┤1     ├┤0     ├──────────────────────────────■─────────────────
     ┌───┐│      ││      │    ┌───┐    ┌────────────────┴────────────────┐
q_1: ┤ H ├┤  Dcx ├┤  Ecr ├────┤ S ├────┤ U(3.4168,2.3054,0.68636,1.6934) ├
     └───┘│      ││      │┌───┴───┴───┐└─────────────────────────────────┘
q_2: ─────┤0     ├┤1     ├┤ P(5.0287) ├───────────────────────────────────
          └──────┘└──────┘└───────────┘

We also draw some random observables 

In [4]:
set_observables = [SparsePauliOp.from_operator(random_hermitian(2**n_qubit)) for _ in range(10)]

For reference, we compute the true final state and the exact expectation values for the different observables.

In [5]:
exp_val = np.zeros((len(qc_random), len(set_observables)), dtype=complex)
for i, qc in enumerate(qc_random):
    exact_state = Statevector(np.array(Operator(qc).data[:,0]).squeeze())
    exp_val[i] = np.array([exact_state.expectation_value(obs) for obs in set_observables])
exp_val = np.real_if_close(exp_val)

# Classical Shadows

We now look at the implementation of Classical Shaodws measurement

In [6]:
# By default, the Classical Shadows (CS) measurement uses X,Y,Z measurements with equal probability.
cs_implementation = ClassicalShadows(n_qubit=n_qubit)
# Define the default shot budget.
cs_shots = 4096

In [7]:
pubs = []
for qcr in qc_random:
    pubs.append((qcr, None, np.random.randint(1000,10000) if np.random.randint(2)==0 else None, cs_implementation))
print(pubs)

[(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x120c886a0>, None, None, <povms.library.pm_sim_implementation.ClassicalShadows object at 0x144b715d0>), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x144b71a80>, None, 5829, <povms.library.pm_sim_implementation.ClassicalShadows object at 0x144b715d0>), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x144b72290>, None, 7400, <povms.library.pm_sim_implementation.ClassicalShadows object at 0x144b715d0>), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x144b723e0>, None, 6648, <povms.library.pm_sim_implementation.ClassicalShadows object at 0x144b715d0>), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x144b718a0>, None, 1239, <povms.library.pm_sim_implementation.ClassicalShadows object at 0x144b715d0>)]


In [8]:
# Run the sampler job locally using AerSimulator.
# Session syntax is supported but ignored because local mode doesn't support sessions.
backend = AerSimulator()
with Session(backend=backend) as session:
    # First define a standard sampler (that will be used under the hood).
    runtime_sampler = Sampler(session=session)
    # Then define the POVM sampler, which takes BaseSampler as an argument.
    sampler = POVMSampler(runtime_sampler)
    # Submit the job by specifying which POVM to use, which circuit(s) to measure and the shot budget.
    cs_job = sampler.run(pubs, shots=cs_shots)

/Users/dao/Documents/Internship/ic-povms/.venv/lib/python3.10/site-packages/qiskit_ibm_runtime/session.py:156: UserWarning: Session is not supported in local testing mode or when using a simulator.
  warnings.warn(


## Results
We retrieve the result object, which contains the POVM used and from which we can query the counts of each outcome.

In [9]:
cs_result = cs_job.result()
print(f"POVM: {cs_result[0].povm}")
print(f"Counts: {cs_result[1].get_counts()}")

POVM: <povms.library.pm_sim_implementation.ClassicalShadows object at 0x144b715d0>
Counts: Counter({(0, 4, 0): 231, (0, 1, 0): 132, (3, 4, 0): 119, (0, 4, 4): 118, (0, 4, 3): 116, (0, 4, 5): 114, (0, 3, 0): 109, (0, 4, 2): 106, (0, 2, 0): 105, (4, 4, 0): 99, (2, 4, 0): 97, (5, 4, 0): 94, (0, 0, 0): 87, (5, 4, 2): 71, (4, 0, 0): 68, (3, 4, 3): 65, (0, 3, 2): 63, (5, 0, 0): 63, (4, 4, 5): 62, (5, 2, 0): 61, (4, 1, 0): 60, (5, 1, 0): 60, (5, 4, 4): 59, (0, 1, 3): 59, (5, 4, 3): 59, (0, 3, 4): 59, (0, 0, 4): 58, (0, 0, 2): 58, (0, 1, 2): 57, (0, 1, 4): 57, (0, 2, 3): 57, (3, 4, 2): 57, (0, 0, 5): 56, (4, 4, 3): 56, (4, 3, 0): 54, (0, 3, 5): 53, (2, 0, 0): 53, (2, 3, 0): 53, (3, 4, 5): 53, (2, 4, 4): 53, (3, 4, 4): 53, (3, 2, 0): 52, (3, 1, 0): 52, (0, 2, 5): 52, (3, 3, 0): 50, (0, 0, 3): 50, (2, 1, 0): 50, (2, 4, 5): 49, (0, 1, 5): 49, (4, 2, 0): 49, (0, 2, 2): 48, (4, 4, 4): 48, (2, 4, 2): 48, (2, 4, 3): 48, (5, 4, 5): 46, (3, 0, 0): 46, (5, 3, 0): 45, (2, 2, 0): 44, (4, 4, 2): 43, (2, 1,

We now define our POVM post-processor, which will use the resukt object to estimate expectation values of some observables.

In [10]:
for i_state in range(len(cs_result)):
    cs_postprocessor = POVMPostprocessor(cs_result[i_state])
    cs_est_exp_val = [cs_postprocessor.get_expectation_value(obs) for obs in set_observables]
    cs_shots = sum(cs_postprocessor.counts.values())

    n = int(np.ceil(np.log10(cs_shots)))
    print('Exact        CS')
    print(f'             ({cs_shots} shots)')
    for i in range(len(set_observables)):
        print(f'{np.real(exp_val[i_state,i]):>10.3e}   {cs_est_exp_val[i]:>{8+n}.3e}   {abs(cs_est_exp_val[i]-np.real(exp_val[i_state,i]))/abs(np.real(exp_val[i_state,i])):>{12-n}.1%}')

Exact        CS
             (4096 shots)
-3.204e-02     -5.146e-02      60.6%
 1.209e+00      1.104e+00       8.7%
-2.340e-01     -1.979e-01      15.4%
-7.448e-03     -9.770e-04      86.9%
-9.002e-01     -8.719e-01       3.1%
 7.029e-01      6.423e-01       8.6%
 2.484e-01      1.964e-01      21.0%
-6.496e-01     -5.846e-01      10.0%
 2.146e-01      1.863e-01      13.2%
 1.567e-01      4.046e-01     158.2%
Exact        CS
             (5829 shots)
-8.858e-01     -8.029e-01       9.4%
 1.306e+00      1.505e+00      15.2%
-5.916e-01     -2.450e-01      58.6%
 1.489e+00      1.467e+00       1.5%
 7.256e-01      8.764e-01      20.8%
 5.334e-01      5.924e-01      11.1%
-1.170e+00     -1.154e+00       1.4%
-1.970e+00     -2.106e+00       6.9%
-2.403e-01      7.572e-03     103.2%
-1.927e-01     -8.127e-02      57.8%
Exact        CS
             (7400 shots)
 4.920e-01      3.872e-01      21.3%
 9.908e-01      9.651e-01       2.6%
 3.389e-01      4.227e-01      24.7%
 8.957e-01      1.038e+

# PM-simulable POVM
We now look at POVM that are simulable (through randomization) with only single-qubit projective measurements (PMs).

In [11]:
pubs_pm = []
for pub in pubs:
    pubs_pm.append((pub[0], None, pub[2], None))

# Define our projective measurements acting on each qubit.
angles = np.array([[0.0, 0.0, 0.5 * np.pi, 0.0, 0.5 * np.pi, 0.5 * np.pi],[1.2,0.0, 0.1,3.14,0.7,0.3], [0.2,0.1, 1.2,3.14,-0.7,0.3]])
# Set the distributions of the shots among the PMs.
bias = np.array([[0.34, 0.33, 0.33], [0.7,0.15,0.15], [0.34,0.33,0.33]])

# Define the PM-simulable POVM.
pmsim_implementation = RandomizedPMs(n_qubit=n_qubit, bias=bias, angles=angles)

In [12]:
pmsim_shots = 4096

with Session(backend=backend) as session:
    # First define a standard sampler (that will be used under the hood).
    runtime_sampler = Sampler(session=session)
    # Then define the POVM sampler, which takes BaseSampler as an argument.
    sampler = POVMSampler(runtime_sampler)
    # Submit the job by specifying which POVM to use, which circuit(s) to measure and the shot budget.
    pmsim_job = sampler.run(pubs_pm, povm=pmsim_implementation, shots=pmsim_shots)

## Results

Retrieve the result of the sampling and use it to estiamte some expectation values via the post-processor

In [13]:
pmsim_result = pmsim_job.result()

for i_state in range(len(pmsim_result)):
    pmsim_postprocessor = POVMPostprocessor(pmsim_result[i_state])
    pmsim_est_exp_val = [pmsim_postprocessor.get_expectation_value(obs) for obs in set_observables]
    pmsim_shots = sum(pmsim_postprocessor.counts.values())

    n = int(np.ceil(np.log10(pmsim_shots)))
    print('Exact        PM-sim')
    print(f'             ({pmsim_shots} shots)')
    for i in range(len(set_observables)):
        print(f'{np.real(exp_val[i_state,i]):>10.3e}   {pmsim_est_exp_val[i]:>{8+n}.3e}   {abs(pmsim_est_exp_val[i]-np.real(exp_val[i_state,i]))/abs(np.real(exp_val[i_state,i])):>{12-n}.1%}')

Exact        PM-sim
             (4096 shots)
-3.204e-02     -2.755e-01     759.8%
 1.209e+00      9.453e-01      21.8%
-2.340e-01      2.018e+00     962.3%
-7.448e-03      1.208e+00   16317.2%
-9.002e-01      1.726e+00     291.8%
 7.029e-01      8.846e-02      87.4%
 2.484e-01     -1.322e+00     632.1%
-6.496e-01     -1.107e+00      70.3%
 2.146e-01      2.358e+00     999.0%
 1.567e-01     -3.376e+00    2254.6%
Exact        PM-sim
             (5829 shots)
-8.858e-01      2.536e+00     386.2%
 1.306e+00      3.919e+00     199.9%
-5.916e-01     -5.347e+00     803.8%
 1.489e+00      5.613e-01      62.3%
 7.256e-01     -2.214e+00     405.1%
 5.334e-01     -1.136e+00     312.9%
-1.170e+00      2.150e-01     118.4%
-1.970e+00     -1.388e-01      93.0%
-2.403e-01     -1.094e+00     355.2%
-1.927e-01     -7.384e-01     283.2%
Exact        PM-sim
             (7400 shots)
 4.920e-01      2.450e+00     397.9%
 9.908e-01      2.855e+00     188.2%
 3.389e-01     -4.577e-01     235.1%
 8.957e-01 